In [ ]:
import requests
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
URL1 = "https://api.icmr.org.in/covid_data/index.php/login"
Credentials = {"username": "subudhi.badri", "password": "subudhi$#*@icmr2022" }

In [ ]:
token = requests.post(url = URL1, json = Credentials).json()['token']

In [ ]:
URL2 = "https://api.icmr.org.in/covid_data/index.php/covid-data"

In [ ]:
def getit(x):
  if (len(str(x)) == 1):
    return "0" + str(x)
  return str(x)

total_cases = []
positive_cases = []
negative_cases = []

In [ ]:
first=True
for month in range(1, 12):      # months upto which we are training the model
  print("Doing for month:", month)
  for date in range(1, 31):
    DATE = "2021-" + getit(month) + "-" + getit(date)
    print("==> Doing for date:", DATE)
    PARAMS2 = {"date_from": DATE + " 00:00:00", "date_to": DATE + " 23:59:59"}
    token = requests.post(url = URL1, json = Credentials).json()['token']
    HEADERS = {"Authorization": "Bearer " + str(token)}
    next=True
    offset_val='0'
    offset=0
  
    while(next==True):

      Duration = {"date_from": DATE + " 00:00:00", "date_to": DATE + " 23:59:59", "offset": offset_val}
      data1=requests.post(url=URL2, json=Duration, headers = HEADERS).json()

      try:
        Total_result=data1['response']['total_result'] 
      except:
        token = requests.post(url = URL1, json = Credentials).json()['token']
        HEADERS = {"Authorization": "Bearer " + str(token)}
        continue

      

      if(Total_result<10000):
        next=False

      if(first==True):
        df=pd.DataFrame(data1['response']['data_result'][:Total_result])
        first=False
      else:
        data=pd.DataFrame(data1['response']['data_result'][:Total_result])
        df=df.append([data])

      offset=offset+Total_result
      offset_val=str(offset)
      

Doing for month: 1
==> Doing for date: 2021-01-26
==> Doing for date: 2021-01-27
==> Doing for date: 2021-01-28
==> Doing for date: 2021-01-29
==> Doing for date: 2021-01-30
==> Doing for date: 2021-01-31


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4166326 entries, 0 to 3366
Data columns (total 24 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   icmr_id                       object
 1   clinical_id                   object
 2   age                           object
 3   age_in                        object
 4   gender                        object
 5   patient_category              object
 6   state_residence               object
 7   district_residence            object
 8   lab_id                        object
 9   date_sample_collection        object
 10  date_sample_received          object
 11  date_of_sample_tested         object
 12  symptoms                      object
 13  underlying_medical_condition  object
 14  testing_kit_used              object
 15  egene_screening               object
 16  ct_value_screening            object
 17  rdrp_confirmatory             object
 18  ct_value_rdrp                 object
 19  orf

In [ ]:
df=df.drop_duplicates()

In [ ]:
compression_opts = dict(method='zip',
                        archive_name='Jan-5.csv')  
df.to_csv('/content/drive/MyDrive/Covid-ICMR-Data/January/Jan-5.zip', index=False,
          compression=compression_opts)  

In [ ]:
'''import zipfile
import pandas as pd
 
# read the dataset using the compression zip
df = pd.read_csv('/content/drive/MyDrive/Covid-ICMR-Data/January/Jan-5.zip',compression='zip')
 
# display dataset
print(df.head())'''

"import zipfile\nimport pandas as pd\n \n# read the dataset using the compression zip\ndf = pd.read_csv('/content/drive/MyDrive/Covid-ICMR-Data/January/Jan-5.zip',compression='zip')\n \n# display dataset\nprint(df.head())"

In [ ]:
df['Date'] = pd.to_datetime(df['entry_date']).dt.date
df['Time'] = pd.to_datetime(df['entry_date']).dt.time

In [ ]:
Final_data=df.groupby(['state_residence','district_residence','Date','final_test_result']).agg({'icmr_id':'count',}).reset_index().rename(columns={'icmr_id':'count'})

In [ ]:
data=Final_data.pivot_table(index=['state_residence','district_residence','Date'], columns=["final_test_result"], values="count", fill_value=0,aggfunc=np.sum)

In [ ]:
data.head()

final_test_result                                          ANTIGEN Negative  \
state_residence             district_residence Date                           
ANDAMAN AND NICOBAR ISLANDS NICOBARS           2021-01-26                 0   
                                               2021-01-27                 0   
                                               2021-01-28                 0   
                                               2021-01-29                 0   
                                               2021-01-30                 0   

final_test_result                                          ANTIGEN Positive  \
state_residence             district_residence Date                           
ANDAMAN AND NICOBAR ISLANDS NICOBARS           2021-01-26                 0   
                                               2021-01-27                 0   
                                               2021-01-28                 0   
                                               2021-01-29                 0   
                                               2021-01-30                 0   

final_test_result                                          Antigen Negative  \
state_residence             district_residence Date                           
ANDAMAN AND NICOBAR ISLANDS NICOBARS           2021-01-26                 0   
                                               2021-01-27                 6   
                                               2021-01-28                 1   
                                               2021-01-29                 7   
                                               2021-01-30                 0   

final_test_result                                          Antigen Positive  \
state_residence             district_residence Date                           
ANDAMAN AND NICOBAR ISLANDS NICOBARS           2021-01-26                 0   
                                               2021-01-27                 0   
                                               2021-01-28                 0   
                                               2021-01-29                 0   
                                               2021-01-30                 0   

final_test_result                                          Inconclusive  \
state_residence             district_residence Date                       
ANDAMAN AND NICOBAR ISLANDS NICOBARS           2021-01-26             0   
                                               2021-01-27             0   
                                               2021-01-28             0   
                                               2021-01-29             0   
                                               2021-01-30             0   

final_test_result                                          Nagative  Negative  \
state_residence             district_residence Date                             
ANDAMAN AND NICOBAR ISLANDS NICOBARS           2021-01-26         0        10   
                                               2021-01-27         0        19   
                                               2021-01-28         0        14   
                                               2021-01-29         0        24   
                                               2021-01-30         0        16   

final_test_result                                          Positive  \
state_residence             district_residence Date                   
ANDAMAN AND NICOBAR ISLANDS NICOBARS           2021-01-26         0   
                                               2021-01-27         0   
                                               2021-01-28         0   
                                               2021-01-29         0   
                                               2021-01-30         0   

final_test_result                                          Rejected (sample leakage)  \
state_residence             district_residence Date                                    
ANDAMAN AND NICO

In [ ]:
data.reset_index(inplace=True)

In [ ]:
data['new_case']=data['ANTIGEN Positive']+data['Antigen Positive']+data['Positive']+data['TrueNAT Screening Positive']

In [ ]:
data['negative_case']=data['Antigen Negative']+data['ANTIGEN Negative']+data['Negative']+ data['TrueNAT Screening Negative']+ data['Nagative']

In [ ]:
data['Failed_test']=data['Inconclusive']+data['Sample Rejected']+data['Spillage']+data['Under Process']+data['Rejected (sample leakage)']

In [ ]:
data['Total_test']=data['Failed_test']+data['negative_case']+data['new_case']

In [ ]:
data.reset_index(drop=True,inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4286 entries, 0 to 4285
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   state_residence             4286 non-null   object
 1   district_residence          4286 non-null   object
 2   Date                        4286 non-null   object
 3   ANTIGEN Negative            4286 non-null   int64 
 4   ANTIGEN Positive            4286 non-null   int64 
 5   Antigen Negative            4286 non-null   int64 
 6   Antigen Positive            4286 non-null   int64 
 7   Inconclusive                4286 non-null   int64 
 8   Nagative                    4286 non-null   int64 
 9   Negative                    4286 non-null   int64 
 10  Positive                    4286 non-null   int64 
 11  Rejected (sample leakage)   4286 non-null   int64 
 12  Sample Rejected             4286 non-null   int64 
 13  Spillage                    4286 non-null   int6

In [ ]:
data.drop(['ANTIGEN Negative','ANTIGEN Positive',	'Antigen Negative',	'Antigen Positive','Nagative','Inconclusive','Negative',	'Positive',	'Rejected (sample leakage)',	'Sample Rejected','Spillage','TrueNAT Screening Negative','TrueNAT Screening Positive',	'Under Process'],axis=1,inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4286 entries, 0 to 4285
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   state_residence     4286 non-null   object
 1   district_residence  4286 non-null   object
 2   Date                4286 non-null   object
 3   new_case            4286 non-null   int64 
 4   Failed_test         4286 non-null   int64 
 5   negative_case       4286 non-null   int64 
 6   Total_test          4286 non-null   int64 
dtypes: int64(4), object(3)
memory usage: 234.5+ KB


In [ ]:

data.to_csv('/content/drive/MyDrive/Covid-ICMR-Data/January/Jan-5-final.csv', index=False)  